In [ ]:
from IPython.core.display import HTML
with open ("../style.css", "r") as file:
    css = file.read()
HTML(css)

# The Knuth-Bendix Completion Algorithm

This notebook presents the [Knuth-Bendix completion algorithm](https://en.wikipedia.org/wiki/Knuth%E2%80%93Bendix_completion_algorithm) for transforming a set of equations into a *confluent* term rewriting system.  This notebook is divided into seven sections.
- [Parsing](#Parsing)
- [Matching](#Matching)
- [Term Rewriting](#Term-Rewriting)
- [Unification](#Unification)
- [The Lexicographic Path Ordering](#The-Lexicographic-Path-Ordering)
- [Critical Pairs](#Critical-Pairs)
- [The Completion Algorithm](#The-Completion-Algorithm)

## Parsing

To begin, we need a parser that is capable of parsing terms and equations. This parser is implemented in `Parser.ipynb` and can parse equations of terms and supports the binary operators `+`, `-`, `*`, `/`, `%`, and `^` (exponentiation) with the usual precedences.  Furthermore, function symbols are supported.  It also provides the function `to_str` for turning terms or equations into strings.  All together, it provides the following functions:
- `parse_file(file_name)` parses a file containing equations between terms.
- `parse_equation(s)` converts the string `s` into an equation.
- `parse_term(s)` converts the string `s` into a term.
- `to_str(o)` converts an object `o` into a string.  The object `o` either is
  * a term,
  * an equation,
  * a list of equations,
  * a set of equations, or
  * a dictionary representing a *substitution*.

Terms and equations are represented as nested tuples.  The parser is implemented using the parser generator [Ply](https://www.dabeaz.com/ply/ply.html).

In [ ]:
%run Parser.ipynb

In [ ]:
t = parse_term('x * y * z')
t

In [ ]:
to_str(t)

In [ ]:
eq = parse_equation('i(x) * x = 1')
eq

In [ ]:
to_str(parse_file('Examples/group-theory-1.eqn'))

[Back to top](#The-Knuth-Bendix-Completion-Algorithm)

## Matching

The function `is_var(t)` checks whether the term `t`is a variable. Variables are represented as nested tuples of the form `($var, name)`, where `name`is the name of the variable.

In [ ]:
def is_var(t):
    return t[0] == '$var'

Given a variable name `x`, the function `make_var(x)` creates a variable with name `x`.

In [ ]:
def make_var(x):
    return ('$var', x)

Given a term `p`, a term `t`,  and a substitution `σ`, the function `match(p, t, σ)` tries to extend the
substitution `σ` so that the equation
$$ p \sigma = t $$
is satisfied.  If this is possible, the function returns `True` and updates the substitution `σ` so that
$p \sigma = t$ holds.  Otherwise, the function returns `False`.

In [ ]:
def match(pattern, term, σ):
    if is_var(pattern):
        _, var = pattern
        if var in σ:
            return σ[var] == term
        else:
            σ[var] = term           # extend σ
            return True
    if pattern[0] == term[0] and len(pattern) == len(term):
        return all(match(pattern[i], term[i], σ) for i in range(1, len(pattern)))

In [ ]:
p = parse_term('i(x) * z')
t = parse_term('i(i(y)) * i(y)')
σ = {}
match(p, t, σ)
to_str(σ)

Given a term `t` (or a set of terms), the function `find_variables(t)` computes the set of all variables occurring in `t`.

In [ ]:
def find_variables(t):
    if isinstance(t, set):
        return { var for term in t
                     for var  in find_variables(term)
               }
    if is_var(t):
        _, var = t
        return { var }
    _, *L = t
    return find_variables_list(L)

Given a list of terms `L` the function `find_variables(L)` computes the set of all variables occurring in `L`.

In [ ]:
def find_variables_list(L):
    if L == []:
        return set()
    return { x for t in L
               for x in find_variables(t)
           }

In [ ]:
eq = parse_equation('(x * y) * z = x * (y * z)')
find_variables(eq)

Given a term `t` and a substitution `σ` that is represented as a dictionary of the form
$$ \sigma = \{ x_1: s_1, \cdots, x_n:s_n \}, $$
the function `apply(t, σ)` computes the term that results from replacing the variables $x_i$ with the terms $s_i$ in `t` for all $i=1,\cdots,n$.  This term is written as $t\sigma$ and if $\sigma = \{ x_1: s_1, \cdots, x_n:s_n \}$, then $t\sigma$ is defined by induction on `t` as follows:
- $x_i\sigma := s_i$,
- $v\sigma := v$ if $v$ is a variable and $v \not\in \{x_1,\cdots,x_n\}$,
- $f(t_1,\cdots,t_n)\sigma := f(t_1\sigma, \cdots, t_n\sigma)$.

In [ ]:
def apply(t, σ):
    "Apply the substitution σ to the term t."
    if is_var(t):
        _, var = t
        if var in σ:
            return σ[var]
        else:
            return t
    else: 
        f, *Ts = t
        return (f,) + tuple(apply(s, σ) for s in Ts)

In [ ]:
p = parse_term('i(x) * x')
t = parse_term('i(i(y)) * i(y)')
σ = {}
match(p, t, σ)
to_str(apply(p, σ))

Given a set of terms or equations `Ts` and a substitution `σ`, the function `apply_set(T, σ)` applies the substitution `σ` to all elements in `Ts`.

In [ ]:
def apply_set(Ts, σ):
    return { apply(t, σ) for t in Ts }

If  $\sigma = \big[ x_1 \mapsto s_1, \cdots, x_m \mapsto s_m \big]$ and
$\tau = \big[ y_1 \mapsto t_1, \cdots, y_n \mapsto t_n \big]$ 
are two substitutions that are <em style="color:blue;">non-overlapping</em>, i.e. such that $\{x_1,\cdots, x_m\} \cap \{y_1,\cdots,y_n\} = \{\}$ holds,
then we define the <em style="color:blue;">composition</em> $\sigma\tau$ of $\sigma$ and $\tau$ as follows:
$$\sigma\tau := \big[ x_1 \mapsto s_1\tau, \cdots, x_m \mapsto s_m\tau,\; y_1 \mapsto t_1, \cdots, y_n \mapsto t_n \big]$$
This definition implies that we the following *associative law* is valid:
$$ x(\sigma\tau) = (x\sigma)\tau $$
The function $\texttt{compose}(\sigma, \tau)$ takes two non-overlapping substitutions and computes the composition $\sigma\tau$.

In [ ]:
def compose(σ, τ):
    Result = { x: apply(s, τ) for (x, s) in σ.items() }
    Result.update(τ)
    return Result

In [ ]:
t1 = parse_term('i(y)')
t2 = parse_term('a * b')
t3 = parse_term('i(b)')
σ = { 'x': t1 }
τ = { 'y': t2, 'z': t3  }
f'compose({to_str(σ)}, {to_str(τ)}) = {to_str(compose(σ, τ))}'

[Back to top](#The-Knuth-Bendix-Completion-Algorithm)

## Term Rewriting

In [ ]:
from string import ascii_lowercase
ascii_lowercase

Given a term `s` and a set of variables `V`, the function `rename_variables(s, V)` renames the variables in `s` so that they differ from the variables in the set `V`.  This will only work if the number of variables occurring in `V` times two is less than the number of letters in the latin alphabet, i.e. less than 26.  Therefore, the set `V` must have at most 13 variables.  For our examples, this is not a restriction.

In [ ]:
def rename_variables(s, Vars):
    assert len(Vars) <= 13, f'Error: too many variables in {Vars}.'
    NewVars = set(ascii_lowercase) - Vars
    NewVars = sorted(list(NewVars))
    σ       = { x: make_var(NewVars[i]) for (i, x) in enumerate(Vars) }
    return apply(s, σ)

In [ ]:
Vars = { 'x', 'y', 'z' }
t = parse_term('x * y * z')
f'rename_variables({to_str(t)}, {Vars}) = {to_str(rename_variables(t, Vars))}'

The function `simplify_step(t, E)` takes two arguments:
- `t` is a term,
- `E` is a set of equations of the form `('=', l, r)`.

The function tries to an equation `l = r` in `E` and a subterm `s` in the term `t` such that the left hand side `l` of the equation matches the subterm `s` using some substitution $\sigma$, i.e. we have $s = l\sigma$.  Then the term `t` is simplified by replacing the subterm `s` in `t` by $r\sigma$.  More formally, if `u` is the position of `s` in `t`, i.e. `t/u = s` then `t` is simplified into the term 
$$ t[u \mapsto r\sigma] $$
If an appropriate subterm `s` is found, the simplified term is returned.  Otherwise, the function returns `None`.

In [ ]:
def simplify_step(t, Equations):
    if is_var(t):
        return None
    for eq in Equations:
        _, lhs, rhs = eq
        σ = {}
        if match(lhs, t, σ):
            return apply(rhs, σ)
    f, *args   = t
    simpleArgs = []
    change     = False
    for arg in args:
        simple = simplify_step(arg, Equations)
        if simple != None:
            simpleArgs += [simple]
            change = True
        else:
            simpleArgs += [arg]
    if change:
        return (f,) + tuple(simpleArgs)
    return None

In [ ]:
E = { parse_equation('(x * y) * z = x * (y * z)') }
t = parse_term('(a * b) * i(b)')
f'simplify_step({to_str(t)}, {to_str(E)}) = {to_str(simplify_step(t, E))}'

The function `normal_form(t, E)` takes a term `t` and and a list (or set) of equations `E` and tries to simplify the term `t` as much as possible using the equations from `E`.  

In the implementation, we have to be careful to rename the variables occurring in `E` so that they are different from the variables occurring in `t`.  Furthermore, we have to take care that we don't identify different variables in `E` by accident.  Therefore, we rename the variables in `E` so that they are both different from the variables in `t` and from the old variables occurring in `E`.

In [ ]:
def normal_form(t, E):
    Vars = find_variables(t) | find_variables(E)
    NewE = []
    for eq in E:
        NewE += [ rename_variables(eq, Vars) ]   
    while True:
        s = simplify_step(t, NewE)
        if s == None:
            return t
        t = s

In [ ]:
l  = parse_term('i(b * a)')
eq = parse_equation('i(a * c) = i(c) * i(a)')
f'normal_form({to_str(l)}, {to_str(eq)}) = {to_str(normal_form(l, {eq}))}'

In [ ]:
E = parse_file('Examples/group-theory-1.eqn')
t = parse_term('(x * i(y)) * y * z')
print(f'normal_form({to_str(t)}, {to_str(E)}) = \n{to_str(normal_form(t, E))}')

Given an equation `eq` and a set of `RewriteRules`, the function `simplify_equations` simplifies the equation `eq` using the given rewrite rules. It returns a pair of terms.

In [ ]:
def simplify_equation(eq, RewriteRules):   
    _, s, t = eq
    new_s = normal_form(s, RewriteRules)
    new_t = normal_form(t, RewriteRules)
    return new_s, new_t

Given a new rewrite `rule`, the function `simplify_rules(RewriteRules, rule)` tries to simplify all rules in `RewriteRules` with `rule`.  If an equation `eq` from `RewriteRules` can be simplified with `rule`, it is further simplified with all rules in `RewriteRules`. 

In [ ]:
def simplify_rules(RewriteRules, rule):
    SimpleEqs = set()
    for eq in RewriteRules:
        _, s, t = eq
        new_s = normal_form(s, { rule })
        if new_s != s:
            new_s = normal_form(new_s, RewriteRules | { rule })
        new_t = normal_form(t, { rule })
        if new_t != t:
            new_t = normal_form(new_t, RewriteRules | { rule })
        if new_s != new_t:
            simple = order_equation(new_s, new_t, Ordering)
            SimpleEqs.add(simple)
        else:
            print(f'removed: {to_str(s)} = {to_str(t)}')    
    return SimpleEqs    

[Back to top](#The-Knuth-Bendix-Completion-Algorithm)

## Unification

In this section, we implement the [unification](https://en.wikipedia.org/wiki/Unification_(computer_science)) algorithm of Martelli and Montanari.

Given a variable name `x` and a term `t`, the function `occurs(x, t)` checks whether `x` occurs in `t`.

In [ ]:
def occurs(x, t):
    if is_var(t):
        _, var = t
        return x == var
    return any(occurs(x, arg) for arg in t[1:])

The algorithm implemented below takes a pair `(E, σ)` as its input.  Here `E` is a set of *syntactical equations* that need to be solved and `σ` is a substitution that is initially empty.  The pair `(E, σ)` is then transformed using the rules of Martelli and Montanari.  The transformation is successful if the pair `(E, σ)` can be transformed into a pair of the form `({}, μ)`.  Then `μ` is the *solution* to the system of equations `E`. 
The rules that can be used to solve a system of *syntactical equations* are as follows:
<ol>
<li> If $y\in\mathcal{V}$ is a variable that does <b style="color:red;">not</b> occur in the term $t$,
     then we perform the following reduction: 
     $$ \Big\langle E \cup \big\{ y \doteq t \big\}, \sigma \Big\rangle \quad\leadsto \quad 
         \Big\langle E[y \mapsto t], \sigma\big[ y \mapsto t \big] \Big\rangle 
     $$
</li>      
<li> If the variable $y$ occurs in the term $t$, then the system of syntactical equations
     $E \cup \big\{ y \doteq t \big\}$ is not solvable:
     $$ \Big\langle E \cup \big\{ y \doteq t \big\}, \sigma \Big\rangle\;\leadsto\; \texttt{None} \quad
        \mbox{if $x \in \textrm{Var}(t)$ and $y \not=t$.}$$
</li>
<li> If $y\in\mathcal{V}$ is a variable and $t$ is no variable, then we use the following rule:
     $$ \Big\langle E \cup \big\{ t \doteq y \big\}, \sigma \Big\rangle \quad\leadsto \quad 
         \Big\langle E \cup \big\{ y \doteq t \big\}, \sigma \Big\rangle.
     $$   
</li>
<li> Trivial syntactical equations of variables can be dropped:
     $$ \Big\langle E \cup \big\{ x \doteq x \big\}, \sigma \Big\rangle \quad\leadsto \quad
         \Big\langle E, \sigma \Big\rangle.
     $$   
</li>
<li> If $f$ is an $n$-ary function symbol, then we have: 
     $$ \Big\langle E \cup \big\{ f(s_1,\cdots,s_n) \doteq f(t_1,\cdots,t_n) \big\}, \sigma \Big\rangle 
         \;\leadsto\; 
         \Big\langle E \cup \big\{ s_1 \doteq t_1, \cdots, s_n \doteq t_n\}, \sigma \Big\rangle.
     $$   
</li>
<li> The system of syntactical equations $E \cup \big\{ f(s_1,\cdots,s_m) \doteq g(t_1,\cdots,t_n) \big\}$
     has <b style="color:red;">no</b> solution if the function symbols $f$ and $g$ are different:
     $$ \Big\langle E \cup \big\{ f(s_1,\cdots,s_m) \doteq g(t_1,\cdots,t_n) \big\},
      \sigma \Big\rangle \;\leadsto\; \texttt{None} \qquad \mbox{if $f \not= g$}.
     $$
</ol>


Given two terms $s$ and $t$, the function $\texttt{unify}(s, t)$ computes the <em style="color:blue;">most general unifier</em> of $s$ and $t$.

In [ ]:
def unify(s, t):
    return solve({('≐', s, t)}, {})

Given a set of <em style="color:blue;">syntactical equations</em> $E$ and a substitution $\sigma$, the function $\texttt{solve}(E, \sigma)$ applies the rules of Martelli and Montanari to solve $E$.

In [ ]:
def solve(E, σ):
    while E != set():
        _, s, t = E.pop()
        if s == t: # remove trivial equations
            continue
        if is_var(s):
            _, x = s
            if occurs(x, t):
                return None
            else: # set x to t
                E = apply_set(E, { x: t })
                σ = compose(σ, { x: t })
        elif is_var(t):
            E.add(('≐', t, s))
        else:
            f    , g     = s[0]      , t[0]
            sArgs, tArgs = s[1:]     , t[1:]
            m    , n     = len(sArgs), len(tArgs)
            if f != g or m != n:
                return None
            else:
                E |= { ('≐', sArgs[i], tArgs[i]) for i in range(m) }
    return σ

In [ ]:
s = parse_term('x * i(x) * (y * z)')
t = parse_term('a * i(1) * b')
f'unify({to_str(s)}, {to_str(t)}) = {to_str(unify(s, t))}'

[Back to top](#The-Knuth-Bendix-Completion-Algorithm)

## The Lexicographic Path Ordering

In order to turn an equations $s = t$ into a rewrite rules, we have to check whether $s$ is more complex that $t$, so that $s$ should be simplified to $t$, or whether $t$ is more complex than $s$ and we should rewrite $t$ into $s$.  To this end, we implement the *lexicographic path ordering*, which is described in detail below.

The function `is_simpler(s, t, D)` receives three arguments.
- `s` and `t` are terms.
- `D` is a dictionary mapping function symbols to <u>different</u> natural numbers.
  We define a total order on function symbols by defining 
  $$ f < g \;\stackrel{_\textrm{def}}{\Longleftrightarrow}\; D[f] < D[g]. $$

The function `is_simpler(s, t, D)` checks whether `s` is *simpler* than `t`.  In order to define this notion, we assume that a total order $<$ is given on the set of function symbols.  Then we define $s \prec t$ (read: `s` is simpler than `t`) inductively via the following cases:
1. $v \prec t$ if $v$ is a variable occurring in $t$ and $v \not = t$.
2. $f(s_1,\cdots,s_m) \prec g(t_1,\cdots,t_n)$ if
   * $f < g$ and 
   * $s_i \prec g(t_1,\cdots,t_n)$ for all $i=1,\cdots, m$.
3. $f(s_1,\cdots,s_m) \prec f(t_1,\cdots,t_m)$ if 
   * there exists an $i \in \{1,\cdots,n\}$ such that $f(s_1,\cdots,s_m) \preceq t_i$ or
   * both of the following conditions are true:
     - $s_i \prec f(t_1,\cdots,t_m)$ for all $i=1,\cdots, m$ and
     - $[s_1, \cdots, s_m] \prec_{\textrm{lex}} [t_1, \cdots, t_m]$.
   
   Here, $\prec_{\textrm{lex}}$ denotes the *lexicographic extension* of the ordering $\prec$ to
   lists of terms.  It is defined as follows:
   $$ [x] + R_1 \prec_{\textrm{lex}} [y] + R_2 \;\stackrel{_\textrm{def}}{\Longleftrightarrow}\;
      x \prec y \,\vee\, \bigl(x = y \wedge R_1 \prec_{\textrm{lex}} R_2\bigr)
   $$
4. $f(s_1,\cdots,s_m) \prec g(t_1,\cdots,t_n)$ if
   * $f > g$ and 
   * there exists an $i \in \{1,\cdots,n\}$ such that $f(s_1,\cdots,s_m) \preceq t_i$.
   
This ordering is known as the *lexicographic path ordering*.

In [ ]:
def is_simpler(s, t, D):
    if is_var(s):
        _, x = s
        return s != t and occurs(x, t)
    if is_var(t):
        return False
    f, *sArgs = s
    g, *tArgs = t
    if D[f] < D[g]:
        return all(is_simpler(arg, t, D) for arg in sArgs)
    if f == g:
        assert len(sArgs) == len(tArgs)
        return any(s == arg or is_simpler(s, arg, D) for arg in tArgs) or \
               all(is_simpler(arg, t, D) for arg in sArgs) and is_simpler_list(sArgs, tArgs, D)
    if D[f] > D[g]:
        return any(s == arg or is_simpler(s, arg, D) for arg in tArgs)
    assert False, f'Error in is_simpler({s}, {t}, {D}): incomplete ordering.'

Given two lists `S` and `T` of terms and a dictionary `D`, the function `is_simpler_list(S, T, D)` checks whether `S` is lexicographically simpler than `T` if the elements of `S` and `T` are compared with the *lexicographical path ordering* $\prec$.  It is assumed that `S` and `T` have the same length.

In [ ]:
def is_simpler_list(S, T, D):
    if S == [] == T:
        return False
    if is_simpler(S[0], T[0], D):
        return True
    if S[0] == T[0]:
        return is_simpler_list(S[1:], T[1:], D)
    return False

In [ ]:
Ordering = { '1': 0, '*': 1, 'i': 2 }

In [ ]:
l = parse_term('(x * y) * z')
r = parse_term('x * (y * z)')
f'is_simpler({to_str(r)}, {to_str(l)}, {Ordering}) = {is_simpler(r, l, Ordering)}'

Given a pair of term `s` and `t` and an `Ordering` of the function symbols occurring in `s` and `t`, the function `order_equation` orders the equation `s = t` with respect to the lexicographic path ordering, i.e. in the ordered equation, the right hand side is simpler than the left hand side.  If `s` and `t` are incomparable, the function raises an exception.

In [ ]:
def order_equation(s, t, Ordering):
    if is_simpler(t, s, Ordering):
        return ('=', s, t)
    elif is_simpler(s, t, Ordering):
        return ('=', t, s)
    else:
        assert False, f'Error: could not order {to_str(s)} = {to_str(t)}'

[Back to top](#The-Knuth-Bendix-Completion-Algorithm)

## Critical Pairs

The central notion of the Knuth-Bendix algorithm is the notion of a *critical pair*.  

Given two equations `lhs1 = rhs1` and `lhs2 = rhs2`, a pair of terms `(s, t)` is a critical pair of these equations if we have the following:
- `u` is a non-trivial position in `lhs1`, i.e. `lhs1/u` is not a variable,
- The subterm `lhs1/u` is unifiable with `lhs2`, i.e.
  $$\mu = \texttt{mgu}(\texttt{lhs}_1 / a, \texttt{lhs}_2) \not= \texttt{None},$$
- $s = \texttt{lhs}_1[a \leftarrow \texttt{rhs}_2]\mu$ and $t = \texttt{rhs}_1\mu$. 

The function `critical_pairs` implemented in this section computes the critical pairs between two rewrite rules. 

Given a term `t`, the function `positions` computes the set $\mathcal{P}os(t)$ of all *positions* in `t` that do not point to variables.  Such positions are called *non-trivial positions*.  Given a term `t`, the set $\mathcal{P}os(t)$ of all positions in $t$ is defined by induction on `t`. 
1. $\mathcal{P}os(v) := \bigl\{()\bigr\} \quad \mbox{if $v$ is a variable} $
2. $\mathcal{P}os\bigl(f(t_0,\cdots,t_{n-1})\bigr) := 
     \bigl\{()\bigr\} \cup 
     \bigl\{ (i,) + u \mid i \in\{0,\cdots,n-1\} \wedge u \in \mathcal{P}os(t_i) \bigr\}
   $
   
Note that since we are programming in Python, positions are zero-based.  Given a position $v$ in a term $t$, we define $t/v$ as the *subterm of $t$ at position $v$* by induction on $t$:
1. $t/() := t$,
2. $f(t_0,\cdots,t_{n-1})/u := t_i/u\texttt{[1:]}$.

In [ ]:
def non_triv_positions(t):
    if is_var(t):
        return set()
    _, *args = t
    Result = { () }
    for i, arg in enumerate(args):
        Result |= { (i,) + a for a in non_triv_positions(arg) }
    return Result

In [ ]:
t = parse_term('x * i(x) * 1')
f'non_triv_positions({to_str(t)}) = {non_triv_positions(t)}'

Given a term `t` and a position `u` in `t`, the function `subterm(t, u)` extracts the subterm that is located at position `u`, i.e. it computes `t/u`. The position `u` is zero-based.

In [ ]:
def subterm(t, u):
    if len(u) == 0:
        return t
    _, *args = t
    i, *ru   = u
    return subterm(args[i], ru)

In [ ]:
t = parse_term('x * i(x) * 1')
f'subterm({to_str(t)}, (0,1)) = {to_str(subterm(t, (0,1)))}'

Given a term `t`, a position `u` in `t` and a term `s`, the function `replace_at(t, u, s)` replaces the subterm at position `u` with `t`.  The position `u` uses zero-based indexing.

In [ ]:
def replace_at(t, u, s):
    if len(u) == 0:
        return s
    i, *ur = u
    f, *args = t
    new_args = []
    for j, arg in enumerate(args):
        if j == i:
            new_args.append(replace_at(arg, ur, s))
        else:
            new_args.append(arg)
    return (f,) + tuple(new_args)

In [ ]:
t = parse_term('x * i(x) * 1')
s = parse_term('a * b') 
f'replace_at({to_str(t)}, (0,1), {to_str(s)}) = {to_str(replace_at(t, (0,1), s))}'

Given two equations `eq1` and `eq2`, the function `critical_pairs(eq1, eq2)` computes the set of all *critical pairs* between these equations.  A pair of terms `(s, t)` is a critical pair of `eq1` and `eq2` if we have
- `eq1` has the form `lhs1 = rhs1`,
- `eq2` has the form `lhs2 = rhs2`,
- `a` is a non-trivial position in `lhs1`,
- $\mu = \texttt{mgu}(\texttt{lhs}_1/a, \texttt{lhs}_2) \not= \texttt{None}$,
- $s = \texttt{lhs}_1[a \leftarrow \texttt{rhs}_2]\mu$ and $t = \texttt{rhs}_1\mu$. 

In [ ]:
def critical_pairs(eq1, eq2):
    Vars = find_variables(eq1)
    eq2  = rename_variables(eq2, Vars) 
    _, lhs1, rhs1 = eq1
    _, lhs2, rhs2 = eq2
    Result = set()
    Positions = non_triv_positions(lhs1)
    for u in Positions:
        if eq1 == eq2 and u == ():
            continue
        s = subterm(lhs1, u)
        𝜇 = unify(s, lhs2)
        if 𝜇 != None:
            lhs1_new = replace_at(lhs1, u, rhs2)
            lhs1_new = apply(lhs1_new, 𝜇)
            rhs1_new = apply(rhs1, 𝜇)
            Result.add( (lhs1_new, rhs1_new) )
    return Result

In [ ]:
eq1 = parse_equation('i(x) * x = 1')
eq2 = parse_equation('(x * y) * z = x * (y * z)')
for s, t in critical_pairs(eq2, eq1):
    print(f'{to_str(s)} = {to_str(t)}')

[Back to top](#The-Knuth-Bendix-Completion-Algorithm)

## The Completion Algorithm

In [ ]:
def print_equations(Equations):
    cnt = 1
    for _, l, r in Equations:
        print(f'{cnt}. {to_str(l)} = {to_str(r)}')
        cnt += 1

Given an equation `eq` of the form `eq = ('=', lhs, rhs)`, the function `complexity(eq)` computes a measure of complexity for the given equation.   This measure of complexity is the length of the string that represents the equation.  This measure of complexity is later used to choose between equations: Less complex equations are more interesting and should be considered first when computing critical pairs.

In [ ]:
def complexity(eq):
    return len(to_str(eq))

In [ ]:
eq = parse_equation('x * i(x) = 1')
complexity(eq)

Given a set of equations `RewriteRules` and a single rewrite rule `eq`, the function `all_critical_pairs(RewriteRules, eq)` computes the set of all *non-trivial critical pairs* that can be build by building critical pairs with an equation from `RewriteRules` and the equation `eq`.  It is assumed that `eq` is already an element of `RewriteRules`.  

In [ ]:
def all_critical_pairs(RewriteRules, eq):
    Result = set()
    for eq1 in RewriteRules:
        Result |= { ('=', l, r) for l, r in critical_pairs(eq1, eq) }
        Result |= { ('=', l, r) for l, r in critical_pairs(eq, eq1) }
    return Result

The module `heapq` provides heap-based priority queues, which are implemented as lists.

In [ ]:
import heapq as hq

Given a file name that contains a set of equations and a dictionary encoding an ordering of the function symbols, the function `knuth_bendix_algorithm` performs the *Knuth-Bendix algorithm*:
1. The given equations are ordered.
2. The ordered equations are pushed into the priority queue `EqtnQue` according to their complexity.
3. The set `RewriteRules` is initialized as the empty set.
4. As long as the priority queue is not empty, the least complex equation `lr` is removed from the 
   priority queue and simplified using the known `RewriteRules`.  
5. If the simplified version of `lr` is non-trivial, all critical pairs between it and the 
   `RewriteRules` are computed.  These critical pairs are pushed onto the priority queue.
6. When no new critical pairs can be found, the set of `RewriteRules` is returned.
   This set is then guaranteed to be a *confluent* set of rewrite rules.

In [ ]:
def knuth_bendix_algorithm(file, Ordering):
    Equations = set()
    Axioms    = set(parse_file(file))
    for _, s, t in Axioms:
        ordered_eq = order_equation(s, t, Ordering)
        Equations.add(ordered_eq)
        print(f'given:   {to_str(ordered_eq[1])} = {to_str(ordered_eq[2])}')
    EqtnQue = []
    for eq in Equations:
        hq.heappush(EqtnQue, (complexity(eq), eq) )
    RewriteRules = set()
    while EqtnQue != []:
        _, lr = hq.heappop(EqtnQue)
        l, r  = simplify_equation(lr, RewriteRules)
        if l != r:
            _, l, r = order_equation(l, r, Ordering)    
            print(f'added:   {to_str(l)} → {to_str(r)}')
            NewEqs = all_critical_pairs(RewriteRules | { ('=', l, r) }, lr)
            for eq in NewEqs:
                s, t = simplify_equation(eq, RewriteRules)
                if s != t:
                    new_rule = order_equation(s, t, Ordering)
                    hq.heappush(EqtnQue, (complexity(new_rule), new_rule) )
            RewriteRules = simplify_rules(RewriteRules, ('=', l, r))
            RewriteRules.add( ('=', l, r) )
    return RewriteRules

In [ ]:
%%time
RewriteRules = knuth_bendix_algorithm('Examples/group-theory-1.eqn', Ordering)
print()
print_equations(RewriteRules)